<a href="https://colab.research.google.com/github/jaredwool/causal_modeling/blob/main/WFH_Causal_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How does Working from Home (WFH) affect productivity?
## A Synthetic Example to illustrate different CAusal approaches that are facilitated through the use of DAGs
This example is based loosely on an example found on the datacamp.com website but I have expanded upon it and added context for clarity

https://www.datacamp.com/tutorial/intro-to-causal-ai-using-the-dowhy-library-in-python

*   List item
*   List item



In [2]:
# install packages
!pip install seaborn statsmodels dowhy lightgbm

  Using cached dowhy-0.11.1-py3-none-any.whl (383 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 3.3 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt

from lightgbm import LGBMRegressor
import statsmodels.api as sm

import dowhy
from dowhy import CausalModel
